In [78]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
raw_file = "/home/ec2-user/SageMaker/Team-5/Fiona/TN_allinone0112.csv"
df = pd.read_csv(raw_file)

/tmp/ipykernel_11566/2699864545.py:2: DtypeWarning: Columns (3,4,5,6,10,15,17,21,22,24,25,27,28,30,31,33,35,36,38,40,41,44,45,48,49,52,53,56,57,60,61,62,63,64,65,66,67,68,69,70,71,72,73,75,78,81,82,83,85,86,94,107,108,109,110,111,112,113,114,115,117,119,122,126,128,129,130,132,133,137,138,142,143,147,148,152,153,157,158,162,163,167,168,169,170,171,172,173,175,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,253,256,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,300,302,303,304,305,307,310,313,317,319,322,324,325,328,330,332,334,335,336,338,343,348,349,351,352,353,354,355,356,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,38

In [80]:
# height_columns = df.columns[df.columns.str.contains('height', case=False)].tolist()
# print("Columns with 'height' in the title:")
# print(height_columns)
# print("")
# weight_columns = df.columns[df.columns.str.contains('weight', case=False)].tolist()
# print("Columns with 'weight' in the title:")
# print(weight_columns)
# print('')
# gluc_columns = df.columns[df.columns.str.contains('gluc', case=False)].tolist()
# print("Columns with 'gluc' in the title:")
# print(gluc_columns)
# print('')

# for column in df.columns:
#     if 'mmol' in column.lower() or 'mgdl' in column.lower():
#         print(f"Unique values of '{column}':")
#         unique_values = df[column].unique()
#         print(unique_values)
#         print('')
        
# for column in df.columns:
#     if any('mmol' in str(value).lower() for value in df[column]):
#         unique_values = df[column].unique()
#         print(f"Unique values in {column}: {unique_values}")
        
# print('')        
# print('Has unit other than mdgl and under 45')        
# for column in df.columns:
#     if 'gluc' in column.lower():
#         numeric_values = pd.to_numeric(df[column], errors='coerce')
#         mask = (numeric_values.shift(1) < 45) & ~df[column].shift(1).astype(str).str.contains('mgdl', case=False)
#         if mask.any():
#             print(column)

In [81]:
df['Heightin_to_cm'] = df['Heightin'] * 2.53999863
df['Height_cm'] = df['Heightin_to_cm'] + df['Heightcm']
df.insert(df.columns.get_loc('Height_cm') + 1, 'Height_cm_units', 'cm')
df.drop(['Heightin_to_cm', 'Heightcm', 'Heightin'], axis=1, inplace=True)

df['Weightlbs_to_kg'] = df['Weightlbs'] * 0.45359237
df['Weight_kg'] = df['Weightlbs_to_kg'] + df['Weightkg']
df.insert(df.columns.get_loc('Weight_kg') + 1, 'Weight_kg_units', 'kg')
df.drop(['Weightlbs_to_kg', 'Weightkg', 'Weightlbs'], axis=1, inplace=True)

In [82]:
for column in df.columns:
    if 'mmol' in column.lower() or 'mgdl' in column.lower():
        new_column_name = column.replace('mmol', '_unit').replace('mgdl', '').replace('Mmoll', '_unit').replace('MgDL', '').replace('mmoll', '_unit')
        df.rename(columns={column: new_column_name}, inplace=True)
        
columns_to_process = ['AvgRecordedFastingGlucoses', 'AvgAllRecordedGlucoses', 'LowestGlucose', 'HighestGlucose']
df[columns_to_process] = df[columns_to_process].apply(lambda x: x.where(x >= 45, x * 18.018))

columns_to_convert = ['AvgRecordedFastingGlucoses_unit', 'AvgAllRecordedGlucoses_unit', 'LowestGlucose_unitl', 'HighestGlucose_unit']
df[columns_to_convert] = df[columns_to_convert].replace('mmol/L', 'mg/dL')

column_to_move1 = 'AvgRecordedFastingGlucoses_unit'
target_position1 = 'AvgRecordedFastingGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move1)
target_index = current_positions.index(target_position1)
new_positions = current_positions[:target_index + 1] + [column_to_move1] + current_positions[target_index + 1:]
df = df[new_positions]

column_to_move2 = 'AvgAllRecordedGlucoses_unit'
target_position2 = 'AvgAllRecordedGlucoses'
current_positions = df.columns.tolist()
current_positions.remove(column_to_move2)
target_index = current_positions.index(target_position2)
new_positions = current_positions[:target_index + 1] + [column_to_move2] + current_positions[target_index + 1:]
df = df[new_positions]

In [83]:
# column_description = df['Height_2'].describe()
# print(column_description)

# specified_column = 'HemoglobinResultsNormal'
# column_index = df.columns.get_loc(specified_column)

# selected_columns = df.iloc[:, max(0, column_index - 1):column_index + 2]

# print(selected_columns)

# mask = (df[specified_column] == 'mmol/L')
# values_to_right = df[specified_column].shift(-1)[mask]
# print(f"Values to the right of cells with 'mmol/L' in {specified_column}:\n{values_to_right}")

# unique_values = df[specified_column].unique()
# print(f"Unique values in {specified_column}:\n{unique_values}")

In [84]:
numeric_columns = df.select_dtypes(include=np.number).columns

columns_to_skip = ['Rbcresults', 'HemoglobinResults']
for column in numeric_columns:
    if column not in columns_to_skip:
        lower_bound = df[column].quantile(0.0025)
        upper_bound = df[column].quantile(0.9975)
        df[column] = np.where((df[column] < lower_bound) | (df[column] > upper_bound), np.nan, df[column])

RBC_filtered_rows = df[df['RBCResultsNormal'] == 'Yes']
Hemoglobin_filtered_rows = df[df['HemoglobinResultsNormal'] == 'Yes']

RBC_condition = 'RBCResultsNormal'
RBC_filter = 'Rbcresults'
filtered_rows = df[df[RBC_condition] == 'Yes'].copy()
lower_bound = filtered_rows[RBC_filter].quantile(0.0025)
upper_bound = filtered_rows[RBC_filter].quantile(0.9975)
filtered_rows.loc[
    (filtered_rows[RBC_filter] < lower_bound) | (filtered_rows[RBC_filter] > upper_bound),
    RBC_filter
] = np.nan

Hemoglobin_condition = 'HemoglobinResultsNormal'
Hemoglobin_filter = 'HemoglobinResults'
filtered_rows = df[df[Hemoglobin_condition] == 'Yes'].copy()
lower_bound = filtered_rows[Hemoglobin_filter].quantile(0.0025)
upper_bound = filtered_rows[Hemoglobin_filter].quantile(0.9975)
filtered_rows.loc[
    (filtered_rows[Hemoglobin_filter] < lower_bound) | (filtered_rows[Hemoglobin_filter] > upper_bound),
    Hemoglobin_filter
] = np.nan

In [85]:
column_to_print = 'HemoglobinResultsNormal'

# Print unique values of the specified column
unique_values = df[column_to_print].unique()
print(unique_values)

[nan 'Yes' 'No']
